# imports

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.applications import Xception, DenseNet201
from tensorflow.keras.callbacks import LearningRateScheduler
import random
from PIL import Image
import shutil

# global variables

In [3]:
# image size options: 192, 224, 311, 512
IMAGE_DIMENSION = 192
VECTOR_LEN = IMAGE_DIMENSION**2
NUM_CLASS = 104

TRAIN_DIR = f'data/jpeg-{IMAGE_DIMENSION}x{IMAGE_DIMENSION}/train'
VAL_DIR = f'data/jpeg-{IMAGE_DIMENSION}x{IMAGE_DIMENSION}/val'
TEST_DIR = f'data/jpeg-{IMAGE_DIMENSION}x{IMAGE_DIMENSION}/test'

BATCH_SIZE = 10
TRAIN_BATCH_SIZE = BATCH_SIZE
VAL_BATCH_SIZE = BATCH_SIZE
TEST_BATCH_SIZE = BATCH_SIZE

# generate datasets

In [4]:
train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
    TRAIN_DIR,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION), 
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True 
)

val_generator = ImageDataGenerator().flow_from_directory(
    VAL_DIR,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
    batch_size=VAL_BATCH_SIZE,
    shuffle=True
)

test_generator = ImageDataGenerator().flow_from_directory(
    TEST_DIR,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
    batch_size=TEST_BATCH_SIZE,
    shuffle=False
)

Found 11459 images belonging to 104 classes.
Found 3710 images belonging to 104 classes.
Found 1294 images belonging to 104 classes.


# preprocessing

### defining functions

In [11]:
def get_subfolder_list(source_folder):
    # returns a list of all the subfolders in the source_folder
    class_list_dir = []

    for file in os.listdir(source_folder):
        d = os.path.join(source_folder, file)
        if os.path.isdir(d):
            class_list_dir.append(d)

    return class_list_dir

def map_classes(subfolder_list):
    # returns a dict with the flower as the key, and (count, directory) as the values
    class_dict = {}

    for class_folder in subfolder_list:
        file_count = sum(len(files) for _, _, files in os.walk(class_folder))
        class_dict[class_folder[24:]] = file_count, class_folder
        
    return class_dict

def get_flower_count(class_dict):
    # returns a list of the number of flowers found in the dict
    flower_count = []

    for flower in list(class_dict.values()):
        flower_count.append(flower[0])

    return flower_count

In [25]:
def get_metrics(flower_count):
    # returns basic metrics when given a list of flower value count
    class_std = np.std(flower_count)
    class_max = max(flower_count)
    class_min = min(flower_count)
    class_mean = np.mean(flower_count)
    class_first_quartile = np.percentile(flower_count, 25)
    class_third_quartile = np.percentile(flower_count, 75)
    class_tenth_percentile = np.percentile(flower_count, 10)
    class_fifth_percentile = np.percentile(flower_count, 5)

    print(f'0. standard deviation: {class_std}')
    print(f'1. max: {class_max}')
    print(f'2. min: {class_min}')
    print(f'3. mean: {class_mean}')
    print(f'4. 25%: {class_first_quartile}')
    print(f'5. 75%: {class_third_quartile}')
    print(f'6. 10%: {class_tenth_percentile}')
    print(f'7. 5%: {class_fifth_percentile}')
    
    return (class_std, class_max, class_min, class_mean, class_first_quartile,
class_third_quartile, class_tenth_percentile, class_fifth_percentile)

### testing above functions

In [26]:
train_subfolders = get_subfolder_list(TRAIN_DIR)
train_dict = map_classes(train_subfolders)
train_flower_count = get_flower_count(train_dict)

In [27]:
get_metrics(train_flower_count)

0. standard deviation: 132.99130714962862
1. max: 707
2. min: 16
3. mean: 111.1826923076923
4. 25%: 30.5
5. 75%: 113.5
6. 10%: 19.0
7. 5%: 17.15


(132.99130714962862, 707, 16, 111.1826923076923, 30.5, 113.5, 19.0, 17.15)

In [28]:
val_subfolders = get_subfolder_list(VAL_DIR)
val_dict = map_classes(val_subfolders)
val_flower_count = get_flower_count(val_dict)

In [29]:
get_metrics(val_flower_count)

0. standard deviation: 42.990624407913046
1. max: 228
2. min: 5
3. mean: 35.69230769230769
4. 25%: 9.0
5. 75%: 37.0
6. 10%: 6.0
7. 5%: 6.0


(42.990624407913046, 228, 5, 35.69230769230769, 9.0, 37.0, 6.0, 6.0)

### more functions

In [ ]:
src = 'data/jpeg-192x192/train/toad lily'
dst = 'data/jpeg-192x192/train_new/toad lily'

def copy_subfolder(source):
    destination = src[:23] + '_new' + src[23:]
    
    shutil.copytree(source, destination, symlinks=False, ignore=None, 
                copy_function=shutil.copy2, ignore_dangling_symlinks=False, dirs_exist_ok=False)
    
def item_count(folder):
    # helper function to identify which folders to remove
    file_count = sum(len(files) for _, _, files in os.walk(folder))
    
    return file_count
    
def select_subset(subfolder_list, percentile):
    for subfolder in subfolder_list:
        if item_count(subfolder) < percentile